#Installing ALLaM

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
allam_model = AutoModelForCausalLM.from_pretrained("ALLaM-AI/ALLaM-7B-Instruct-preview")
tokenizer = AutoTokenizer.from_pretrained("ALLaM-AI/ALLaM-7B-Instruct-preview")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.03G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

In [9]:
messages=[
    {"role": "user", "content": "اريدك ان تكتب قصيدة شعر "},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] اريدك ان تكتب قصيدة شعر [/INST] حبّيتَك من يومِ شفتَك، وعرفتَك،

وجيتِك كانَ نورَ العيونِ،

وقلبي صارَ ينبضُ بحُبَّك،

وما أقدر أعيشَ من دونَك.

يا أحلىَ ما في الدنيا،

يا نورَ العينِ والروحِ،

يا منبعَ الفرحِ والسعادةِ،

يا منْ جعلتني أعيشُ في حُلمٍ.

أحبَّك يا أغلىَ الناسِ،

وأريدَك دومًا في حياتي،

يا منْ تسكنُ في قلبي،

وتجعلُني أعيشُ في سعادةٍ.

فأنا أُحبكَ بكلِّ مشاعري،

وأعتزُّ بكَ في كلِّ لحظةٍ،

وأريدُ أن نبقىَ معًا،

في حياةٍ مليئةٍ بالسعادةِ.

فيا حبيبي، يا منْ تُضيءُ لي حياتي،

أرجوكَ أن نبقىَ معًا،

فأنا أُحبكَ بكلِّ قلبي،

وأريدكَ دومًا في حياتي. 


#Installing

In [10]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 73.5 MB/s eta 0:00:00


In [17]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

In [14]:
import pickle
from google.colab import drive
drive.mount('/content/drive') # Mount Google Drive to /content/drive

with open("/content/drive/My Drive/embeddings.pkl", "rb") as file:
    data = pickle.load(file)

df = pd.DataFrame(data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2').to('cuda')

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
def make_index() -> faiss.Index:
    data = np.stack(df['combined_embedding'].to_numpy())
    norms = np.linalg.norm(data, axis=1, keepdims=True)  # Compute L2 norm for each vector
    normalized_vectors = data / norms  # Divide each vector by its norm
    index = faiss.IndexFlatIP(2304)  # Inner product (cosine similarity)
    index.add(normalized_vectors)  # Add normalized data to index
    return index

def make_query(title: str, verses: str, theme: str, index: faiss.Index, k=3):
    embedding_1 = embedder.encode(title)
    embedding_2 = embedder.encode(verses)
    embedding_3 = embedder.encode(theme)
    all_embedding = np.concatenate([embedding_1, embedding_2, embedding_3])

    faiss.normalize_L2(all_embedding.reshape(1, -1))  # Normalize query vector
    distances, indices = index.search(all_embedding.reshape(1, -1), k)
    return distances[0], indices[0]

In [23]:
index = make_index()

In [24]:
distances, indices = make_query('الام', '', 'قصيدة مدح', index)

In [25]:
for d, i in zip(distances, indices):
    print(f"Distance: {d}, Index: {i}")
    print(df.iloc[i]['poem title'])
    print(df.iloc[i]['poem verses'])
    print(df.iloc[i]['poem theme'])


Distance: 0.6819446682929993, Index: 917
أماه
['أمّاهُ كمْ أرهقتنا دُرْبةُ الأوّاهْ', 'مذ فَلَّقَ الدمعُ في أرواحِنا مَجْراهْ', 'في جفنِ عينيكِ باتَ الحُزنُ مُعتكفًا', 'ورَجْعُ حُزنُك في قلبي بنى مَبناهْ', 'الهمُّ جمرٌ بنا أذْكى حَرائِقَهُ', 'وحظّنا تَرِبٌ والفَقْدُ قَدْ أرْواهْ', 'قَوْسُ الرَّزايا تمادى في تَصيُّدِنا', 'وسَعدُنا قَدْ غَفا أوّاهُ ما أشْقاهْ', 'تناسَلَ الويْلُ في أوْصالِنا زَمنًا', 'والبينُ يا "مِيْمَتيْ " في ركبِنا مَسْراهْ', 'في وحْشَة الليلُ أحبابٌ لنا رحَلوا', 'النَّوحُ لازمَنا ، للفَقْدِ ما أمْلاهْ', 'تَطُوفُ روحُ أبي تَسْعى تُهدهدنا', 'يصْحو الحنينُ جوًى اذْ نجْتَبي ذِكْراهْ', 'تحومُ روحُ أخي والفَجْعُ يشربُنا', 'نُخْفي توجُّعَنا في حَشْرجاتِ الآهْ', 'هذا جَبينُكِ أطْلالٌ لذاكِرتي', 'عَليْهِ خَطَّ الضَّنَى مَرْثيةً تَنْعاهْ', 'الكَوْنُ عتْمٌ وتاهَت فيهِ وجْهتُنا', 'تَغْريبةٌ نحْنُ والعِنْوانُ ضَيّعْناهْ', 'أنامُ في جُبِّ كابوسٍ يُروِّعُني', 'حُشاشَتي تَصْطلي: لا تَرْحلي أمّاهْ']
قصيدة عامه
Distance: 0.6074661016464233, Index: 6281
يا أم نوفل فكي عانيا مثلت
['يا أُ

#RAG

In [26]:
llm_input = 'اريد من كتابتك قصيدة عن الحب ولا تكرر'
distances, indices = make_query('غرام', '', '', index)
f"{llm_input}\n\قصائد مشابهه:\n{df.iloc[indices[1]]['poem verses']}"



NameError: name 'llm_input' is not defined

In [31]:
for d, i in zip(distances, indices):
    print(f"Distance: {d}, Index: {i}")
    print(df.iloc[i]['poem title'])
    print(df.iloc[i]['poem verses'])
    print(df.iloc[i]['poem theme'])


Distance: 0.6941684484481812, Index: 20507
 لذ بالكريم وسل منه الرضا كرما
[' لذ بــالكــريــم وسـل مـنـه الرضـا كـرمـا', 'إن الكــريــم إذا اســتــرحــمــتــه رحـمـا', ' واجــعــل شــفــيـعـك مـدح الهـاشـمـي فـمـا', 'جـــاب امـــرؤ فـــاغـــر بـــالمـــدح فــمــا', ' ولا تــقــم شــاهــدا إلا الخــلوص عــســى', 'يـكـفـيـك مـا قـد بـدا مـنـه ومـا انكتما', ' ولا تــمــل لهــوى الدنــيــا وزخــرفــهــا', 'فـــعـــن قـــريـــب تــرى وجــدانــه عــدمــا', ' هـــي الغـــرور فـــكـــم غــرت أخــا ثــقــة', 'بــــعــــهــــدا ولوت عـــن وعـــده عـــدمـــا', ' وهــي القــتــول فـكـم أفـنـت فـتـى عـلقـت', 'بـــذيـــلهــا يــده واســتــســمــنــت ورمــا', ' لم يــــصــــف مـــوردهـــا يـــومـــا لوارده', 'إلا وصـــــيـــــره وقــــع الدلا وخــــمــــا', ' ولا غـــدا بـــهـــواهـــا صــبــهــا كــلفــا', 'إلا وأعــقــبــهــا الخــســران والنــدمــا', ' فــاحــذر هـواهـا واحـذر نـهـج فـتـنـتـهـا', 'فـهـي التـي مـا سـقـت إلا أعـقـبـت بـظما', ' وآربــا بــنــفــســك عــن أوخــام لذتــهــا', 'فـهـي ال

In [32]:
llm_input = 'اريد من كتابتك قصيدة عن الحب وقافية تنتهي بالياء'
distances, indices = make_query('غرام', '', '', index)
f"{llm_input}\n\قصائد مشابهه:\n{df.iloc[indices[1]]['poem verses']}"


messages=[
    {"role": "user", "content": f"{llm_input}\n\قصائد مشابهه:\n{df.iloc[indices[1]]['poem verses']}"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] اريد من كتابتك قصيدة عن الحب وقافية تنتهي بالياء
\قصائد مشابهه:
[' يـا رَبْـعُ فـيـكَ وسـيـمُ البِـشْـرِ مُـجْـتـمِـعُ', 'وقـــد تـــفَّرقَ عـــن أجـــزاعـــك الجَـــزَعُ', ' إِيــهــاً لكَ الخـيـرُ مـن رَبْـعٍ فـلا كـدَرٌ', 'يُـــرى بـــصــفــوكَ مــصــطــافٌ ومــرتــبــعُ', ' حــيَّاــك كـفُّ الحَـيـا مُـسْـحَـنْـفِـراً ديـمـاً', 'فــالخــصْــبُ مُــلْتــمـعٌ والجـدبُ مُـمـتـنِـعُ', ' وللخـــليـــطِ انـــشـــراحٌ لا يـــدافـــعُهُ', 'عُــسْــرٌ وفــي راحــتـيْهِ اليُـسْـرُ مُـنْـدفِـعُ', ' فـانـفُـضْ لنـا زهـرَك الزاهـي كفيتَ أسىً', 'فــالعــيــش مــنــخــفِــضٌ والحـظُّ مـرتـفـعُ', ' والوُرْقُ فــي مـنْـبَـرِ الأغـصـانِ خـاطـبـةٌ', 'والروضُ جــامــعُهــا والبــهـجـةُ الجُـمَـعُ', ' وذاتُ خــال شَــمُــوعٍ مــن مــحــاســنــهــا', 'إِذا تـــغَـــطَّشـــَ ليـــلٌ يــوقَــدُ الشــمَــعُ', ' قــد أكْــمَــلتْ بــعــد عـشـرٍ سـنُّهـا سَـنَـةٌ', 'فَــطــرْفُهــا لحَــشَــا أهــلِ الهــوى سَـبُـعُ', ' تُـنـازعُ الْبَـانَ مَـيْـسـاً فـهْـيَ إِن خَـطَـرتْ', 'تــخــالُهــا بــدلالِ الغ

In [33]:
llm_input = 'اريد قصيدة ب4 ابيات عن الحب'
distances, indices = make_query('غرام', '', '', index)
f"{llm_input}\n\قصائد مشابهه:\n{df.iloc[indices[1]]['poem verses']}"


messages=[
    {"role": "user", "content": f"{llm_input}\n\قصائد مشابهه:\n{df.iloc[indices[1]]['poem verses']}"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] اريد قصيدة ب4 ابيات عن الحب
\قصائد مشابهه:
[' يـا رَبْـعُ فـيـكَ وسـيـمُ البِـشْـرِ مُـجْـتـمِـعُ', 'وقـــد تـــفَّرقَ عـــن أجـــزاعـــك الجَـــزَعُ', ' إِيــهــاً لكَ الخـيـرُ مـن رَبْـعٍ فـلا كـدَرٌ', 'يُـــرى بـــصــفــوكَ مــصــطــافٌ ومــرتــبــعُ', ' حــيَّاــك كـفُّ الحَـيـا مُـسْـحَـنْـفِـراً ديـمـاً', 'فــالخــصْــبُ مُــلْتــمـعٌ والجـدبُ مُـمـتـنِـعُ', ' وللخـــليـــطِ انـــشـــراحٌ لا يـــدافـــعُهُ', 'عُــسْــرٌ وفــي راحــتـيْهِ اليُـسْـرُ مُـنْـدفِـعُ', ' فـانـفُـضْ لنـا زهـرَك الزاهـي كفيتَ أسىً', 'فــالعــيــش مــنــخــفِــضٌ والحـظُّ مـرتـفـعُ', ' والوُرْقُ فــي مـنْـبَـرِ الأغـصـانِ خـاطـبـةٌ', 'والروضُ جــامــعُهــا والبــهـجـةُ الجُـمَـعُ', ' وذاتُ خــال شَــمُــوعٍ مــن مــحــاســنــهــا', 'إِذا تـــغَـــطَّشـــَ ليـــلٌ يــوقَــدُ الشــمَــعُ', ' قــد أكْــمَــلتْ بــعــد عـشـرٍ سـنُّهـا سَـنَـةٌ', 'فَــطــرْفُهــا لحَــشَــا أهــلِ الهــوى سَـبُـعُ', ' تُـنـازعُ الْبَـانَ مَـيْـسـاً فـهْـيَ إِن خَـطَـرتْ', 'تــخــالُهــا بــدلالِ الغُــنْــجِ تــنــتــزع

In [34]:
llm_input = 'اريد 4 ابيات عن الحب'
distances, indices = make_query('غرام', '', '', index)
f"{llm_input}\n\قصائد مشابهه:\n{df.iloc[indices[1]]['poem verses']}"


messages=[
    {"role": "user", "content": f"{llm_input}\n\قصائد مشابهه:\n{df.iloc[indices[1]]['poem verses']}"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] اريد 4 ابيات عن الحب
\قصائد مشابهه:
[' يـا رَبْـعُ فـيـكَ وسـيـمُ البِـشْـرِ مُـجْـتـمِـعُ', 'وقـــد تـــفَّرقَ عـــن أجـــزاعـــك الجَـــزَعُ', ' إِيــهــاً لكَ الخـيـرُ مـن رَبْـعٍ فـلا كـدَرٌ', 'يُـــرى بـــصــفــوكَ مــصــطــافٌ ومــرتــبــعُ', ' حــيَّاــك كـفُّ الحَـيـا مُـسْـحَـنْـفِـراً ديـمـاً', 'فــالخــصْــبُ مُــلْتــمـعٌ والجـدبُ مُـمـتـنِـعُ', ' وللخـــليـــطِ انـــشـــراحٌ لا يـــدافـــعُهُ', 'عُــسْــرٌ وفــي راحــتـيْهِ اليُـسْـرُ مُـنْـدفِـعُ', ' فـانـفُـضْ لنـا زهـرَك الزاهـي كفيتَ أسىً', 'فــالعــيــش مــنــخــفِــضٌ والحـظُّ مـرتـفـعُ', ' والوُرْقُ فــي مـنْـبَـرِ الأغـصـانِ خـاطـبـةٌ', 'والروضُ جــامــعُهــا والبــهـجـةُ الجُـمَـعُ', ' وذاتُ خــال شَــمُــوعٍ مــن مــحــاســنــهــا', 'إِذا تـــغَـــطَّشـــَ ليـــلٌ يــوقَــدُ الشــمَــعُ', ' قــد أكْــمَــلتْ بــعــد عـشـرٍ سـنُّهـا سَـنَـةٌ', 'فَــطــرْفُهــا لحَــشَــا أهــلِ الهــوى سَـبُـعُ', ' تُـنـازعُ الْبَـانَ مَـيْـسـاً فـهْـيَ إِن خَـطَـرتْ', 'تــخــالُهــا بــدلالِ الغُــنْــجِ تــنــتــزعُ', ' ق

#No RAG

In [36]:
messages=[
    {"role": "user", "content": "اريد قصيدة عن الام"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] اريد قصيدة عن الام [/INST] أمي يا شمساً في سماء حياتي

وأحلى من العسل في حياتي

أمي يا منبع الحنان والوفا

أنتِ الحب الذي لا يُقدر بثمن

أمي يا نبع الخير والعطاء

أنتِ القلب الذي يحتضنني

أمي يا منارة الأمل والضياء

أنتِ النور الذي يضيء دربي

أمي يا ملاكي الطاهر

أنتِ الروح التي تنبض في جسدي

أمي يا شمساً في سماء حياتي

وأحلى من العسل في حياتي

أمي يا منبع الحنان والوفا

أنتِ الحب الذي لا يُقدر بثمن

أمي يا نبع الخير والعطاء

أنتِ القلب الذي يحتضنني

أمي يا منارة الأمل والضياء

أنتِ النور الذي يضيء دربي

أمي يا ملاكي الطاهر

أنتِ الروح التي تنبض في جسدي

أمي يا منبع الحنان والوفا

أنتِ الحب الذي لا يُقدر بثمن

أمي يا نبع الخير والعطاء

أنتِ القلب الذي يحتضنني

أمي يا منارة الأمل والضياء

أنتِ النور الذي يضيء دربي

أمي يا ملاكي الطاهر

أنتِ الروح التي تنبض في جسدي

أمي يا منبع الحنان والوفا

أنتِ الحب الذي لا يُقدر بثمن

أمي يا نبع الخير والعطاء

أنتِ القلب الذي يحتضنني

أمي يا منارة الأمل والضياء

أنتِ النور الذي يضيء دربي

أمي يا ملاكي الطاهر

أنتِ الروح التي تنبض في جسدي

أمي 

In [38]:
messages=[
    {"role": "user", "content": "اكتب بيت شعر على بحر الكامل يبدأ بـ (يا ليل)"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print('')
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[INST] اكتب بيت شعر على بحر الكامل يبدأ بـ (يا ليل) [/INST] يا ليلُ هلْ لأبْدى السكونِ حكايةً  
 تُشْفي جروحَ القلبِ منْ آلامِهِ  
  
أمْ هلْ لأبْدى الليلُ الشجيُّ صدى  
  
منْ عاشقٍ يشكو الهوى بلْهامهِ  
  
أمْ هلْ لأبْدى الليلُ الجريحُ حكايةً  
  
تُنسِي الحزانى في دروبِ ظلامِهِ  
  
يا ليلُ هلْ تسقي القلوبَ بريقَ  
  
منْ نورِ الحقيقةِ في أحلامِهِ 


In [39]:
messages=[
    {"role": "user", "content": "اكتب قصيدة عربية فصيحة مكونة من 4 أبيات عن الصداقة، على وزن البحر الكامل، مع التزام القافية في نهاية كل بيت"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print('')
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[INST] اكتب قصيدة عربية فصيحة مكونة من 4 أبيات عن الصداقة، على وزن البحر الكامل، مع التزام القافية في نهاية كل بيت [/INST] لي صديق في وده لي صدوق،  
وصون للعهد مني رفيقي  
شيمته الوفاء وإن جفيته،  
فقل لغيره كيف الطريق 


In [40]:
messages=[
    {"role": "user", "content": "اكتب أبياتًا شعرية بأسلوب المتنبي تصف الشجاعة، مستخدمًا ألفاظًا جزلة وتراكيب معقدة، مع التزام القافية والوزن الشعري"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print('')
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[INST] اكتب أبياتًا شعرية بأسلوب المتنبي تصف الشجاعة، مستخدمًا ألفاظًا جزلة وتراكيب معقدة، مع التزام القافية والوزن الشعري [/INST] أُعِيذُكِ الرَيفَ لا تَهَبِي فَخُذي

بَأسَ الرِجالِ عَلى الأَمرِ الجَمِّ جَمِي

أَرى المَرءَ مِثلَ السَيفِ إِن ضَرَبَت بِهِ

يَنفُصِ الزِندُ بِه وَيَقتُلُه النِصلُ

فَلا تَنثَني عَنكِ النُفوسُ بِرَغمِها

إِذا كُنتِ في الأَمرِ الجَمِّ جَمِي

وَما أَنتَ إِلّا البَدرُ في ضَوءِهِ

يَقتُلُ الظَلامَ وَهوَ في السَدمِ

فَإِن كُنتَ في غَمرةٍ فَاِنهَض بِها

فَإِنَّ التَعاويذَ لِلرَوعِ تَقتَدي

فَإِنَّ الجَنانَ لَيسَ لَها مَدى

إِلّا بِفَتحِ البَأسِ في كُلِّ صَعِد

فَإِنَّ النُفوسَ لِرَغمِ الخُطوبِ

تُضيءُ بِشَمسِ الشَجاعةِ في السَمَد

فَإِنَّ الشَجاعَةَ تَفَكُّ الحَديدَ

وَتَقتُلُ الرَوعَ في كُلِّ نَفسٍ

فَلا تَخافي الدُجى إِن أَقبَلَ النَجمُ

فَإِنَّ النَهارَ سَيَقتُلُ الظَلمَ 


In [41]:
input = 'اكتب قصيدة عربية مكونة من 6 أبيات تمدح القيم النبيلة مثل الكرم والشجاعة. يجب أن يكون لكل بيت شطران متساويان في الوزن، وتنتهي جميع الأبيات بنفس القافية'

messages=[
    {"role": "user", "content": input},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print('')
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[INST] اكتب قصيدة عربية مكونة من 6 أبيات تمدح القيم النبيلة مثل الكرم والشجاعة. يجب أن يكون لكل بيت شطران متساويان في الوزن، وتنتهي جميع الأبيات بنفس القافية [/INST] في ساحة المجد، كنتَ الأولَ في النَدى
وفي الشجاعةِ كنتَ الأولَ في الحَملِ

عَطاؤُكَ في الوغى، جَسرٌ لناجٍ،
وجُهدُكَ في العَلا، جَسرٌ لناجٍ

حُلمُكَ كالنَّجم، يَهدي العُلا،
وشُجاعتُكَ كالرَعد، تَقتُلُ العِللا

يا فارسَ الحقِّ، يا رمزَ الفِداءِ،
أنتَ في القلبِ، يا رمزَ النُبلِ والعطاءِ 


In [42]:
input = 'اكتب قصيدة من 6 أبيات تصف جمال الطبيعة في فصل الربيع، متضمنة صورًا شعرية عن الأزهار، النسيم، والأنهار. يجب أن يكون لكل بيت شطران متساويان في الوزن وتنتهي جميع الأبيات بنفس القافية'

messages=[
    {"role": "user", "content": input},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print('')
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[INST] اكتب قصيدة من 6 أبيات تصف جمال الطبيعة في فصل الربيع، متضمنة صورًا شعرية عن الأزهار، النسيم، والأنهار. يجب أن يكون لكل بيت شطران متساويان في الوزن وتنتهي جميع الأبيات بنفس القافية [/INST] في فصل الربيع، تنبض الأرض بالحياة
تزهر الأزهار، تسحر الألباب في المساء
نسيم عليل، يلامس الأغصان بلطف
وتنحني الأنهار، كأنها ترقص في وقف

تغرد الطيور، في سماواتٍ زرقاء
وتغني الطبيعة، بألحانٍ عذراء
كل شيء ينبض، في هذا الزمان
جمال الربيع، في كل مكان

يا له من فصل، يسحر القلب والروح
حيث الجمال، ينسج في كل وضوح 


In [43]:
input = 'اكتب أبيات تعبر عن السعي وراء الأحلام والطموحات، مع التشبيه بالطائر الذي يحلق في السماء. يجب أن تكون الأبيات موزونة ولها قافية موحدة'

messages=[
    {"role": "user", "content": input},
]
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print('')
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[INST] اكتب أبيات تعبر عن السعي وراء الأحلام والطموحات، مع التشبيه بالطائر الذي يحلق في السماء. يجب أن تكون الأبيات موزونة ولها قافية موحدة [/INST] حلّق كطائر في السماء،
لا تجعل اليأس يأخذك مأخذًا،
فالطموح هو الدواء،
لتعبر كل الحواجز.

لا تدع الخوف يحدّك،
أو الفشل يثنيك،
فكل خطوة هي خطوة،
لتحقق أحلامك.

سعى بكل جهد،
ولا تتوقف،
فالسعي هو الطريق،
لتحقق ما تريد.

استمر في الصعود،
ولا تتخلى،
فالنجاح هو الهدف،
والطموح هو النور.

اجعل من كل يوم،
فرصة لتحقيق،
ما كنت تحلم به،
ولا تتردد في السير.

فالحلم لا يموت،
طالما أنت حي،
والطموح لا ينطفئ،
طالما القلب ينبض. 
